In [ ]:
pip install --upgrade yfinance

In [ ]:
import yfinance as yf
import pandas as pd

# Define the stock symbol and time period
stock_symbol = "AMZN"
start_date = "2022-01-01"
end_date = "2023-01-01"

# Download historical stock data
stock_data = yf.download(stock_symbol, start=start_date, end=end_date)

# Print the first few rows of the data
print(stock_data.head())

In [7]:
US_stocks=pd.read_csv('/Users/elbezaharon/Desktop/3A ENSAE/AML/Project/Data/nasdaq_screener_1701944424423.csv')
US_stocks_tickets=list(US_stocks[US_stocks['Market Cap']>20000000000]['Symbol'])

In [8]:
len(US_stocks_tickets)

502

In [9]:
date_index = pd.date_range(start='2013-01-01', end='2023-11-21', freq='D')

# Create an empty DataFrame with the datetime index
empty_df = pd.DataFrame(index=date_index)
empty_df['offvalue']=1
dfs=[empty_df]
for symbol in US_stocks_tickets:

# Make API request
    data = pd.DataFrame(yf.download(symbol, start='2013-01-01', end='2023-11-21', interval='1d'))
    if not data.empty:
        df=data.rename(columns={'Adj Close':f'{symbol}'})
        df=df[f'{symbol}']
        dfs+=[df]
    else:
        pass
        

from functools import reduce
stocks_closes_data = reduce(lambda df1,df2: pd.merge(df1,df2,left_index=True,right_index=True, how='outer'), dfs)
stocks_closes_data=stocks_closes_data.drop('offvalue',axis=1)
null_values=pd.DataFrame(stocks_closes_data.isnull().sum())
columns_no_null=null_values[null_values[0]<1300].index

final_stocks_closes_data=stocks_closes_data[columns_no_null]
final_stocks_closes_data=final_stocks_closes_data.dropna(axis = 0, how = 'all')
final_stocks_closes_data
final_stocks_closes_data.to_csv('/Users/elbezaharon/Desktop/3A ENSAE/AML/Project/Data/US_stocks_full_data.csv',index=False)
final_stocks_closes_data.head()

KeyboardInterrupt: 

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
final_stocks_closes_data=pd.read_csv('/Users/elbezaharon/Desktop/3A ENSAE/AML/Project/Data/US_stocks_full_data.csv',index_col=0)
null_values=pd.DataFrame(final_stocks_closes_data.isnull().sum())
columns_no_null=null_values[null_values[0]<1300].index



returns_data=final_stocks_closes_data.sort_index().pct_change().dropna()[:100]
reshaped_returns = pd.melt(returns_data.reset_index(), id_vars='index', var_name='Stock', value_name='return')
reshaped_returns=reshaped_returns.rename(columns={'index':'Date'})
reshaped_returns['rank']=reshaped_returns.groupby('Date')['return'].rank()

X=reshaped_returns.drop('rank',axis=1)
y=reshaped_returns['rank']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [1]:
import numpy as np
from tqdm import tqdm
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import pandas as pd


final_stocks_closes_data=pd.read_csv('/Users/elbezaharon/Desktop/3A ENSAE/AML/Project/Data/US_stocks_full_data.csv',index_col=0)
null_values=pd.DataFrame(final_stocks_closes_data.isnull().sum())
columns_no_null=null_values[null_values[0]<1300].index

final_stocks_closes_data=final_stocks_closes_data[columns_no_null]
final_stocks_closes_data=final_stocks_closes_data.dropna(axis = 0, how = 'all')
final_stocks_closes_data


returns_data=final_stocks_closes_data.sort_index().pct_change().dropna()[:100]
reshaped_returns = pd.melt(returns_data.reset_index(), id_vars='index', var_name='Stock', value_name='return')
reshaped_returns=reshaped_returns.rename(columns={'index':'Date'})
reshaped_returns['rank']=reshaped_returns.groupby('Date')['return'].rank()


learning_rate = 0.1

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def lambda_loss_gradient(y_true, y_pred, p_lambda, delta=1.0):
    S_ij = np.sign(y_true.values.flatten()[:, None] - y_true.values.flatten())
    gradient = np.sum((1 / (1 + np.exp(y_pred - y_pred[:, None]))) - S_ij, axis=1)
    return gradient

def lambda_mart(X_train, y_train, num_trees=10, learning_rate=0.1):
    n = len(y_train)
    trees = []

    p_lambda = np.sign(y_train.values[:, None] - y_train.values)

    scores = np.zeros(n)

    for _ in range(num_trees):
        gradient = lambda_loss_gradient(y_train, scores, p_lambda)

        tree = DecisionTreeRegressor(max_depth=3)
        tree.fit(X_train, -gradient)

        scores += learning_rate * tree.predict(X_train)
        trees.append(tree)

    return trees

# Example usage:

X=reshaped_returns.drop('rank',axis=1)
y=reshaped_returns['rank']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train_encoded = pd.get_dummies(X_train)
X_test_encoded=pd.get_dummies(X_test)
# Train LambdaMART
ensemble = lambda_mart(X_train_encoded, y_train)

# Make predictions on new data
def predict_lambda_mart(ensemble, X):
    return np.sum(learning_rate * tree.predict(X) for tree in ensemble)

# Example usage:
predictions = [predict_lambda_mart(ensemble, x.reshape(1, -1)) for x in X_test_encoded.values]
print(predictions)


/var/folders/gf/shnqljgn16z6q38qcsr75kwm0000gn/T/ipykernel_12346/3416969910.py:30: RuntimeWarning: overflow encountered in exp
  gradient = np.sum((1 / (1 + np.exp(y_pred - y_pred[:, None]))) - S_ij, axis=1)
/var/folders/gf/shnqljgn16z6q38qcsr75kwm0000gn/T/ipykernel_12346/3416969910.py:30: RuntimeWarning: overflow encountered in exp
  gradient = np.sum((1 / (1 + np.exp(y_pred - y_pred[:, None]))) - S_ij, axis=1)
/var/folders/gf/shnqljgn16z6q38qcsr75kwm0000gn/T/ipykernel_12346/3416969910.py:30: RuntimeWarning: overflow encountered in exp
  gradient = np.sum((1 / (1 + np.exp(y_pred - y_pred[:, None]))) - S_ij, axis=1)
/var/folders/gf/shnqljgn16z6q38qcsr75kwm0000gn/T/ipykernel_12346/3416969910.py:30: RuntimeWarning: overflow encountered in exp
  gradient = np.sum((1 / (1 + np.exp(y_pred - y_pred[:, None]))) - S_ij, axis=1)
/var/folders/gf/shnqljgn16z6q38qcsr75kwm0000gn/T/ipykernel_12346/3416969910.py:30: RuntimeWarning: overflow encountered in exp
  gradient = np.sum((1 / (1 + np.exp(y_pr

In [ ]:
from sklearn.metrics import accuracy_score

X_test['Pred']=predictions
X_test['Pred_rank']=X_test.groupby("Date")['Pred'].rank()
accuracy_score(X_test['Pred_rank'].astype(int),y_test.astype(int))

In [32]:
X_test

,Date,Stock,return,Pred,Pred_rank
36506,2013-02-12,TFC,-0.000979,[5762.705218117304],72.5
2265,2013-05-08,AME,0.013353,[3961.2861079219288],22.0
30324,2013-03-11,PKX,-0.008026,[6352.605704264353],106.5
7481,2013-05-31,CCEP,-0.014847,[6961.4303965863455],71.0
30545,2013-04-10,PM,0.007616,[5190.989945329214],113.5
...,...,...,...,...,...
912,2013-02-21,ADM,-0.012000,[6961.4303965863455],89.0
22599,2013-06-26,LMT,0.020256,[3229.8567285908475],9.5
9302,2013-02-06,COP,-0.002252,[6352.605704264353],116.5
14710,2013-02-19,ETN,0.010186,[4593.571553784861],45.5


0.0019251336898395723

In [40]:
accuracy_score(X_test['Pred_rank'].astype(int),y_test.astype(int))

0.0019251336898395723